In [1]:
import pandas as pd
import pickle

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# **A bit data preps**

In [7]:
data = pd.read_csv("../datasets/dataset-iob-tag.csv",encoding="latin1")
data.head()

,sentence_id,words,labels
0,0,NOTE,O
1,1106,Very,O
2,1106,bad,O
3,1106,experience,O
4,1106,with,O


In [6]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"])
data

KeyError: 'Sentence #'

In [5]:
data.rename(columns={"Sentence #": "sentence_id", "Word": "words", "Tag": "labels"}, inplace =True)
data["labels"] = data["labels"].str.upper()

In [8]:
data.dropna(inplace=True)

In [9]:
data.isna().any()

sentence_id    False
words          False
labels         False
dtype: bool

In [10]:
X = data[["sentence_id","words"]]
Y = data["labels"]

In [11]:
label = ['O',
 'B-STORAGE',
 'B-PERFORMANCE',
 'B-BATTERY',
 'I-BATTERY',
 'B-CAMERA',
 'I-CAMERA',
 'B-CHARGING',
 'I-CHARGING',
 'I-STORAGE',
 'I-PERFORMANCE',
 'B-SCREEN',
 'I-SCREEN',
 'B-AUDIO',
 'B-FEATURE',
 'I-FEATURE',
 'B-CONNECTION',
 'I-CONNECTION',
 'B-BUTTON',
 'I-BUTTON',
 'I-AUDIO']

# **Training**

In [12]:
from simpletransformers.ner import NERModel, NERArgs
import torch.nn as nn
args = NERArgs()
args.num_train_epochs = 9
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [14]:
model = NERModel('distilbert', 'dslim/distilbert-NER', labels=label, args=args, ignore_mismatched_sizes=True)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dslim/distilbert-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([21, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
train_data = pd.DataFrame({"sentence_id": x_train["sentence_id"], "words": x_train["words"], "labels":y_train})
test_data = pd.DataFrame({"sentence_id": x_test["sentence_id"], "words": x_test["words"], "labels":y_test})

In [17]:
model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 1 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

Running Epoch 2 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

Running Epoch 3 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

...

Running Epoch 6 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

Running Epoch 7 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

Running Epoch 8 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

Running Epoch 9 of 9:   0%|          | 0/152 [00:00<?, ?it/s]

(1368, 0.04271507338914186)

In [18]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

In [19]:
result

{'eval_loss': 0.04449694887388475,
 'precision': 0.9544740973312402,
 'recall': 0.9310872894333844,
 'f1_score': 0.9426356589147287}

In [20]:
prediction, model_output = model.predict(["fingerprint reader recognises and unlocks mobile"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
prediction

[[{'fingerprint': 'B-FEATURE'},
  {'reader': 'I-FEATURE'},
  {'recognises': 'O'},
  {'and': 'O'},
  {'unlocks': 'O'},
  {'mobile': 'O'}]]

In [46]:
with open('model/NER_Model_pickle_v4', 'wb') as file:
  pickle.dump(model, file)

In [ ]:
# kf = StratifiedKFold(n_splits=10)
# for train, test in kf.split(X, Y):
#   train_data = pd.DataFrame({"sentence_id": X.iloc[train]["sentence_id"], "words": X.iloc[train]["words"], "labels":Y.iloc[train]})
#   test_data = pd.DataFrame({"sentence_id": X.iloc[test]["sentence_id"], "words": X.iloc[test]["words"], "labels":Y.iloc[test]})
#   model.train_model(train_data,eval_data = test_data,acc=accuracy_score)